In [1]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import matplotlib as mpl

from Functions import DateTime
from FinRockDataFeeder import PdDataFeeder
from FinRockTradingEnv import TradingEnv
from FinRockEnv import PygameRender
from FinRockMinMax import MinMaxScaler
from FinRockObservations import simpleReward

In [2]:
df = pd.read_csv("/home/zymantas/Desktop/Training_data/1HData/ATOMUSDT1H.csv")

pd_data_feeder  = PdDataFeeder(df)

In [3]:
env = TradingEnv(
    data_feeder = pd_data_feeder,
    output_transformer = MinMaxScaler(min = pd_data_feeder.min, max = pd_data_feeder.max),
    initial_balance = 1000.0,
    max_episode_steps = 1000,
    window_size = 50,
    reward_function = simpleReward
)

action_space = env.action_space

pygameRender = PygameRender(frame_rate = 10)

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
state, info = env.reset()
pygameRender.render(info)
rewards = 0.0

while True:
    # Simulate model prediction, now use random action
    action = np.random.randint(0, action_space)
    # 0 action always hold

    state, reward, terminated, truncated, info = env.step(action)
    rewards += reward
    pygameRender.render(info)

    if terminated or truncated:
        print(info['states'][-1].account_value, rewards)
        rewards = 0.0
        state, info = env.reset()
        pygameRender.reset()

955.2313902461079 0.10381847561694847
985.1186595723201 0.02316056616445151
1088.7783145438182 -0.07059278388032647
962.0089742300738 -0.09063190700515554
995.4162036204549 -0.24776995946446756
1355.1368518171316 0.3361660363213452
1479.1023586752144 0.6006351918483399
695.5325007111047 -0.49492156997490144
970.6969559829639 -0.04687433448877707
1205.4129844896786 0.31580006180614195
833.2272728129103 -0.34112688234578536
853.3784740982898 -0.24902965652219056
850.403053454245 -0.11175238710002042
1125.9067655149072 0.25204887055858294
929.1056026872019 -0.022229903710929377
933.6603623508585 -0.019285692847116048
763.2454484408711 -0.27480099211373055
902.359085524231 0.02790386151707116
1023.507718517883 -0.26410005481921955
964.0162695700744 0.03585841165625324
958.9153017953759 -0.11555942539240456
790.0091622597384 -0.296539809917827
1263.5096842675202 0.37564662153308376
902.144760846097 -0.10553731204358746
1055.4671032131866 0.32240034656396155
945.833190636227 0.01498639361391

KeyboardInterrupt: 